# Análise Exploratória - Sistema de Restaurante

Este notebook demonstra análises exploratórias dos dados gerados pelo sistema de streaming.

In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Conexão com Banco de Dados

In [ ]:
# Configuração de conexão
conn_params = {
    'host': 'localhost',
    'database': 'datastore',
    'user': 'admin',
    'password': 'password',
    'port': 5432
}

# Conectar ao banco
conn = psycopg2.connect(**conn_params)
print("Conexão estabelecida com sucesso!")

## Análise dos Pedidos

In [ ]:
# Carregar dados de pedidos
query_pedidos = """
SELECT 
    nome_prato,
    preco,
    quantidade,
    cliente_id,
    created_at,
    (preco * quantidade) as valor_total
FROM pedidos 
ORDER BY created_at DESC
LIMIT 10000
"""

df_pedidos = pd.read_sql(query_pedidos, conn)
df_pedidos['created_at'] = pd.to_datetime(df_pedidos['created_at'])

print(f"Total de pedidos carregados: {len(df_pedidos)}")
df_pedidos.head()

In [ ]:
# Estatísticas descritivas
print("=== ESTATÍSTICAS GERAIS ===")
print(f"Período: {df_pedidos['created_at'].min()} até {df_pedidos['created_at'].max()}")
print(f"Total de pedidos: {len(df_pedidos):,}")
print(f"Valor total: R$ {df_pedidos['valor_total'].sum():,.2f}")
print(f"Ticket médio: R$ {df_pedidos['valor_total'].mean():.2f}")
print(f"Clientes únicos: {df_pedidos['cliente_id'].nunique()}")
print(f"Pratos únicos: {df_pedidos['nome_prato'].nunique()}")

In [ ]:
# Top 10 pratos mais pedidos
top_pratos = df_pedidos.groupby('nome_prato').agg({
    'quantidade': 'sum',
    'valor_total': 'sum',
    'id': 'count'
}).rename(columns={'id': 'num_pedidos'}).sort_values('quantidade', ascending=False)

print("=== TOP 10 PRATOS MAIS PEDIDOS ===")
print(top_pratos.head(10))

## Visualizações

In [ ]:
# Gráfico de pratos mais populares
plt.figure(figsize=(12, 6))
top_10_pratos = top_pratos.head(10)
plt.bar(range(len(top_10_pratos)), top_10_pratos['quantidade'])
plt.xticks(range(len(top_10_pratos)), top_10_pratos.index, rotation=45, ha='right')
plt.title('Top 10 Pratos Mais Pedidos')
plt.ylabel('Quantidade Total')
plt.tight_layout()
plt.show()

In [ ]:
# Distribuição de preços
plt.figure(figsize=(10, 6))
plt.hist(df_pedidos['preco'], bins=20, alpha=0.7, edgecolor='black')
plt.title('Distribuição de Preços dos Pratos')
plt.xlabel('Preço (R$)')
plt.ylabel('Frequência')
plt.axvline(df_pedidos['preco'].mean(), color='red', linestyle='--', label=f'Média: R$ {df_pedidos["preco"].mean():.2f}')
plt.legend()
plt.show()

In [ ]:
# Vendas por hora
df_pedidos['hora'] = df_pedidos['created_at'].dt.hour
vendas_por_hora = df_pedidos.groupby('hora')['valor_total'].sum()

plt.figure(figsize=(12, 6))
plt.plot(vendas_por_hora.index, vendas_por_hora.values, marker='o')
plt.title('Vendas por Hora do Dia')
plt.xlabel('Hora')
plt.ylabel('Valor Total (R$)')
plt.grid(True, alpha=0.3)
plt.show()

## Análise dos Relatórios Spark

In [ ]:
# Carregar relatórios do Spark
query_relatorios = """
SELECT 
    periodo,
    total_vendas,
    prato_mais_vendido,
    quantidade_total,
    ticket_medio,
    created_at
FROM relatorio_vendas 
ORDER BY created_at DESC
LIMIT 1000
"""

df_relatorios = pd.read_sql(query_relatorios, conn)
df_relatorios['created_at'] = pd.to_datetime(df_relatorios['created_at'])
df_relatorios['periodo'] = pd.to_datetime(df_relatorios['periodo'])

print(f"Total de relatórios: {len(df_relatorios)}")
df_relatorios.head()

In [ ]:
# Evolução das vendas ao longo do tempo
plt.figure(figsize=(15, 8))

# Subplot 1: Total de vendas
plt.subplot(2, 2, 1)
plt.plot(df_relatorios['created_at'], df_relatorios['total_vendas'])
plt.title('Evolução do Total de Vendas')
plt.ylabel('Valor (R$)')
plt.xticks(rotation=45)

# Subplot 2: Quantidade total
plt.subplot(2, 2, 2)
plt.plot(df_relatorios['created_at'], df_relatorios['quantidade_total'], color='orange')
plt.title('Evolução da Quantidade Total')
plt.ylabel('Quantidade')
plt.xticks(rotation=45)

# Subplot 3: Ticket médio
plt.subplot(2, 2, 3)
plt.plot(df_relatorios['created_at'], df_relatorios['ticket_medio'], color='green')
plt.title('Evolução do Ticket Médio')
plt.ylabel('Valor (R$)')
plt.xticks(rotation=45)

# Subplot 4: Pratos mais vendidos
plt.subplot(2, 2, 4)
prato_counts = df_relatorios['prato_mais_vendido'].value_counts().head(5)
plt.pie(prato_counts.values, labels=prato_counts.index, autopct='%1.1f%%')
plt.title('Pratos Mais Vendidos (Frequência nos Relatórios)')

plt.tight_layout()
plt.show()

## Insights e Conclusões

In [ ]:
print("=== INSIGHTS PRINCIPAIS ===")
print(f"1. Prato mais popular: {top_pratos.index[0]} ({top_pratos.iloc[0]['quantidade']} unidades)")
print(f"2. Maior faturamento: {top_pratos.sort_values('valor_total', ascending=False).index[0]}")
print(f"3. Preço médio dos pratos: R$ {df_pedidos['preco'].mean():.2f}")
print(f"4. Quantidade média por pedido: {df_pedidos['quantidade'].mean():.1f}")
print(f"5. Cliente mais ativo: {df_pedidos['cliente_id'].value_counts().index[0]} ({df_pedidos['cliente_id'].value_counts().iloc[0]} pedidos)")

if len(df_relatorios) > 0:
    print(f"6. Média de vendas por minuto: R$ {df_relatorios['total_vendas'].mean():.2f}")
    print(f"7. Prato mais frequente nos relatórios: {df_relatorios['prato_mais_vendido'].mode()[0]}")

In [ ]:
# Fechar conexão
conn.close()
print("Análise concluída!")